# Preparing a ligand with `openmmforcefields` and Interchange

[`openmmforcefields`](https://github.com/openmm/openmmforcefields) is a Python package that provides OpenMM implementations of some small molecule force fields via small molecule template generators. Its key utility is providing classes that port SMIRNOFF, GAFF, and Espaloma force fields into a format that be used alongside the many force fields that ship with OpenMM by default.

## Comparing to GAFF parametrization

`openmmforcefields` provides a (partial) implementation of GAFF/GAFF2, using AM1-BCC in place of RESP to generate partial charges. The API for using `GAFFTemplateGenerator` is similar to `SMIRNOFFTemplateGenerator`.

In [ ]:
import openmm.app
from openff.toolkit import Molecule
from openff.units import ensure_quantity, unit
from openmmforcefields.generators import GAFFTemplateGenerator

molecule = Molecule.from_smiles("O=S(=O)(N)c1c(Cl)cc2c(c1)S(=O)(=O)NCN2")
molecule.generate_conformers(n_conformers=1)

topology = molecule.to_topology()
topology.box_vectors = unit.Quantity([4, 4, 4], unit.nanometer)

gaff = GAFFTemplateGenerator(molecules=molecule)
forcefield_gaff = openmm.app.ForceField()
forcefield_gaff.registerTemplateGenerator(gaff.generator)

system_gaff = forcefield_gaff.createSystem(
    topology=topology.to_openmm(),
    nonbondedMethod=openmm.app.PME,
)

In [ ]:
%env INTERCHANGE_EXPERIMENTAL=1

From here, one could use `from_openmm` to import this `openmm.System` into an `Interchange` object, which then could be combined with other `Interchange` objects generated by different paths.

In [ ]:
from openff.interchange.interop.openmm import from_openmm

imported = from_openmm(
    molecule.to_topology().to_openmm(),
    system_gaff,
    box_vectors=None,
    positions=ensure_quantity(molecule.conformers[0], "openmm"),
)

We can use Interchange's machinery to evaluate each of these objects - the `openmm.System` created by `openmmforcefields` and the `Interchange` object created from it.

In [ ]:
from openff.interchange.drivers.openmm import (
    _get_openmm_energies,
    _process,
    get_openmm_energies,
)

energy_openmmforcefields = _process(
    _get_openmm_energies(
        system_gaff,
        box_vectors=None,
        positions=ensure_quantity(molecule.conformers[0], "openmm"),
        round_positions=None,
        platform="Reference",
    ),
    system=system_gaff,
    combine_nonbonded_forces=False,
    detailed=False,
)

energy_openmmforcefields

In [ ]:
energy_imported = get_openmm_energies(imported)

energy_imported

Except for some small differences in non-bonded energies due to PME errors, the energies evaluated from each of these objects should match quite closely.

In [ ]:
energy_imported.compare(
    energy_openmmforcefields,
    {"Nonbonded": 0.002 * unit.kilojoule_per_mole},
)

energy_imported.diff(energy_openmmforcefields)